## Pytorch-Lightning学习笔记

pytorch-lighting（简称pl），它其实就是一个轻量级的PyTorch库，用于高性能人工智能研究的轻量级PyTorch包装器。缩放你的模型，而不是样板。  
Pytorch-lightning可以非常简洁得构建深度学习代码。但是其实大部分人用不到很多复杂得功能。而pl有时候包装得过于深了，用的时候稍微有一些不灵活。通常来说，在你的模型搭建好之后，大部分的功能都会被封装在一个叫trainer的类里面。一些比较麻烦但是需要的功能通常如下, 通过pl就可以很好的实现：  
* 保存checkpoints
* 输出log信息
* resume training 即重载训练，我们希望可以接着上一次的epoch继续训练
* 记录模型训练的过程(通常使用tensorboard)
* 设置seed，即保证训练过程可以复制

实现流程：
* 初始化 def __ init __(self) 
* 训练training_step(self, batch, batch_idx) 
* 校验validation_step(self, batch, batch_idx) 
* 测试 test_step(self, batch, batch_idx)

## LightningModuel
LightningModule将你的PyTorch代码组织成6个部分:
* initialization (__ init__ and setup())：初始化
* Train Loop (training_step())：训练
* Validation Loop (validation_step())：验证
* Validation Loop (validation_step())：测试
* Prediction Loop (predict_step())：预测
* Optimizers and LR Schedulers (configure_optimizers())：优化器和学习率调整程序

框架：  
net = MyLightningModuleNet()  
trainer = Trainer()  
trainer.fit(net)  

## 简单实现


In [35]:
# 导包
import os

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F

### __ init __（初始化）
构造函数中，像常见的 torch.nn.Module 一样，我们定义好模型的层。由于是最简实例，这里只有一层线性层，将手写数字图像映射为输出 logits。

### forward（前向传播）
由于是继承自 torch.nn.Module，因此实现 forward 方法是必须的。forward 方法要完成模型的前向过程，这里直接调用 __ init__ 中定义好的线性层，完成模型前向过程。

### train_dataloader（训练数据集获取）
train_dataloader 方法也是最简实现中必须的，它的功能是获取训练集的 DataLoader。这里我们返回 MNIST 数据集的 DataLoader。dataloader 的获取也可以不在类内实现，而是在 fit 时传入，后面会介绍。

### training_step（训练步骤）
training_step 是是 LigtningModule 的核心方法，它定义了一个训练步中需要做的事情。在深度学习的训练步中，最核心的事情就是模型前向，得到结果，计算损失，反向传播，更新参数，这几步在 pytorch 中都有对应的方法供调用。但是在 pytorch lightning 中，我们只需要进行模型前向，并返回必要的信息即可。在最简实现中，我们只需返回损失。


### configure_optimizer（配置优化器）
在 training_step 中，我们只需返回损失，这意味着模型的反向传播和参数更新过程由 pytorch lightning 帮我们完成了。虽然这个过程可以有框架自己完成，但是我们还是要指定参数更新所用的优化器，在很多模型中，优化器、学习率等超参数设置对结果影响很大。在最简实现中，我们设置好学习率，并返回一个 Adam 优化器。

In [36]:
# 简易的pl模型
class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)


In [37]:
# 开始训练
train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())) # 获取数据
trainer = pl.Trainer(max_epochs=2) #初始化训练器
model = LitModel() #初始化模型

trainer.fit(model, train_dataloaders=train_loader) #开始训练

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


以上代码实现了一个最简的 pytorch lightning 训练过程。这足以体现出 pytorch lightning 的简洁、易用。但是，显然这个最简实现缺少了很多东西，比如验证、测试、日志打印、模型保存等。接下来，我们将实现相对完整但依旧简洁的 pytorch lightning 模型开发过程。

## 补充
我们还缺少验证和测试的步骤：
* 模型定义 _ init_
* 前向计算 forward
* 训练/ 验证/ 测试 （training_step/ validation_step/ test_step）
* 训练/ 验证/ 测试结束后（training_step_end/validation_step_end/test_step_end）
* 选用优化器 (configure_optimizers)
* 数据加载器 (train_dataloader, val_dataloader, test_dataloader)

### LightningModule自带工具
* log：Tensorboard 损失/指标日志保存和查看，不要自己定义，直接用即可。用法非常简单，将要记录的值传入:self.log('train loss', loss)
* print:LightningModule 提供的 print 只打印一次
* freeze:冻结所有权重以供预测时候使用。仅当已经训练完成且后面只测试时使用



### 拓展实例


In [38]:
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import numpy as np


class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.fc(x.view(-1, 28 * 28)))

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return {'loss': loss}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        correct = pred.eq(y.view_as(pred)).sum().item()
        acc = correct / x.shape[0]
        self.log('val_acc', acc, on_step=False, on_epoch=True)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return {'val_loss': loss, 'val_acc': acc}

   

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return {'test_loss': loss}

   

    def configure_optimizers(self):
        # REQUIRED
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        # REQUIRED
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor()), batch_size=32)

    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=32)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=32)

model = MNISTModel()
trainer = pl.Trainer(
        max_epochs=10,
        callbacks=[pl.callbacks.EarlyStopping( monitor="val_loss", patience=3)]
)
trainer.fit(model)
trainer.test()


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | fc   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Restoring states from the checkpoint path at d:\Github_code\Deep-learning\ex\lightning_logs\version_9\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at d:\Github_code\Deep-learning\ex\lightning_logs\version_9\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

[{}]

## 参考
[pytorch lightning最简上手](https://blog.csdn.net/weixin_44966641/article/details/127827124)  
[Pytorch Lightning 完全攻略](https://zhuanlan.zhihu.com/p/319810661)  
[pytorch-lightning入门](https://blog.csdn.net/u014264373/article/details/117021901)